<img src="../../../images/qiskit_header.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" align="middle">

# Quantum Process Tomography

* **Last Updated:** June 17, 2019
* **Requires:** qiskit-terra 0.8, qiskit-ignis 0.1.1, qiskit-aer 0.2

This notebook contains examples for using the ``ignis.verification.tomography`` process tomography module.

In [1]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.quantum_info import state_fidelity, process_fidelity
from qiskit.tools.qi.qi import outer

# Tomography functions
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter

## 1-qubit process tomography example

In [2]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=4000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp',), ('X',)): {'0': 4000},
 (('Zp',), ('Y',)): {'0': 2033, '1': 1967},
 (('Zp',), ('Z',)): {'0': 1985, '1': 2015},
 (('Zm',), ('X',)): {'1': 4000},
 (('Zm',), ('Y',)): {'0': 1959, '1': 2041},
 (('Zm',), ('Z',)): {'0': 2026, '1': 1974},
 (('Xp',), ('X',)): {'0': 2030, '1': 1970},
 (('Xp',), ('Y',)): {'0': 1993, '1': 2007},
 (('Xp',), ('Z',)): {'0': 4000},
 (('Yp',), ('X',)): {'0': 2040, '1': 1960},
 (('Yp',), ('Y',)): {'1': 4000},
 (('Yp',), ('Z',)): {'0': 1979, '1': 2021}}

In [3]:
# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 2, choi_lstsq.data / 2))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi_lstsq.data, require_cptp=False)))


# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 2, choi_cvx.data / 2))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi_cvx.data, require_cptp=False)))


Least-Sq Fitter
fit time: 0.009121894836425781
fit fidelity (state): 0.9976767994222256
fit fidelity (process): 0.995358994837865

CVXOPT Fitter
fit time: 0.05771899223327637
fit fidelity (state): 0.9999108703920029
fit fidelity (process): 0.9998217487227923


## 1-qubit process tomography of two-qubit swap gate

We will prepare qubit-0 and measure qubit-1 so the reconstructed channel should be an identity

In [4]:
# Process tomography of a Hadamard gate
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.swap(q[0], q[1])

# Ideal channel is a unitary
ideal_unitary = np.eye(2)
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
# We use the optional prepared_qubits kwarg to specify that the prepared qubit was different to measured qubit
qpt_circs = process_tomography_circuits(circ, q[1], prepared_qubits=q[0])
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp',), ('X',)): {'0': 1007, '1': 993},
 (('Zp',), ('Y',)): {'0': 962, '1': 1038},
 (('Zp',), ('Z',)): {'0': 2000},
 (('Zm',), ('X',)): {'0': 1059, '1': 941},
 (('Zm',), ('Y',)): {'0': 1000, '1': 1000},
 (('Zm',), ('Z',)): {'1': 2000},
 (('Xp',), ('X',)): {'0': 2000},
 (('Xp',), ('Y',)): {'0': 1010, '1': 990},
 (('Xp',), ('Z',)): {'0': 995, '1': 1005},
 (('Yp',), ('X',)): {'0': 1003, '1': 997},
 (('Yp',), ('Y',)): {'0': 2000},
 (('Yp',), ('Z',)): {'0': 978, '1': 1022}}

In [5]:
# Least-Squares tomographic reconstruction
t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 2, choi_lstsq.data / 2))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi_lstsq.data, require_cptp=False)))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 2, choi_cvx.data / 2))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi_cvx.data, require_cptp=False)))

Least-Sq Fitter
fit time: 0.006497859954833984
fit fidelity (state): 0.9866920175150853
fit fidelity (process): 0.9735611374279911

CVXOPT Fitter
fit time: 0.04198098182678223
fit fidelity (state): 0.999940174723841
fit fidelity (process): 0.9998803530267468


## 2-Qubit entangling circuit

In [6]:
# Bell-state entangling circuit
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.h(q[0])
circ.cx(q[0], q[1])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)

t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 4, choi_lstsq.data / 4))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi_lstsq.data, require_cptp=False)))

t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 4, choi_cvx.data / 4))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi_cvx.data, require_cptp=False)))

Least-Sq Fitter
fit time: 0.1802959442138672
fit fidelity (state): 0.9842194915019892
fit fidelity (process): 0.9686880063880137

CVXOPT Fitter
fit time: 1.1846849918365479
fit fidelity (state): 0.9999324749848556
fit fidelity (process): 0.9998649545043932


## Using SIC-POVM preparation basis

In [7]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q, prep_labels='SIC', prep_basis='SIC')
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs, prep_basis='SIC')
qpt_tomo.data

{(('S0',), ('X',)): {'0': 2000},
 (('S0',), ('Y',)): {'0': 999, '1': 1001},
 (('S0',), ('Z',)): {'0': 1013, '1': 987},
 (('S1',), ('X',)): {'0': 713, '1': 1287},
 (('S1',), ('Y',)): {'0': 977, '1': 1023},
 (('S1',), ('Z',)): {'0': 1938, '1': 62},
 (('S2',), ('X',)): {'0': 665, '1': 1335},
 (('S2',), ('Y',)): {'0': 1828, '1': 172},
 (('S2',), ('Z',)): {'0': 543, '1': 1457},
 (('S3',), ('X',)): {'0': 645, '1': 1355},
 (('S3',), ('Y',)): {'0': 206, '1': 1794},
 (('S3',), ('Z',)): {'0': 507, '1': 1493}}

In [8]:
# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 2, choi_lstsq.data / 2))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi_lstsq.data, require_cptp=False)))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')

print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 2, choi_cvx.data / 2))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi_cvx.data, require_cptp=False)))

Least-Sq Fitter
fit time: 0.007508039474487305
fit fidelity (state): 0.9906085840000641
fit fidelity (process): 0.9813053636480525

CVXOPT Fitter
fit time: 0.04143118858337402
fit fidelity (state): 0.9958380801102131
fit fidelity (process): 0.9916934801256183
